In [1]:
import json
import pandas
import pathlib
import pydash
import requests

def value_extract(row, column):

    ''' Extract dictionary values. '''
    
    return pydash.get(row[column], 'value')

def sparql_query(query, service):

    ''' Send sparql request, and formulate results into a dataframe. '''

    response = requests.get(service, params={'format': 'json', 'query': query}, timeout=120)
    results = pydash.get(response.json(), 'results.bindings')
    df = pandas.DataFrame.from_dict(results)
    for column in df.columns:
        df[column] = df.apply(value_extract, column=column, axis=1)
    
    return df

def acmi_holdings(wikidata_id):

    ''' Pull ACMI holdings from public API. '''

    query = '''
        select distinct ?wikidata_id ?acmi_id where {
            values ?wikidata_id {'''+' '.join(['wd:'+x for x in wikidata_id])+'''}
            ?wikidata_id wdt:P7003 ?acmi_id
            } '''

    acmi_tsv = pandas.read_csv('https://raw.githubusercontent.com/ACMILabs/acmi-api/main/app/tsv/works.tsv', delimiter='\t')
    acmi_tsv['acmi_id'] = 'works/'+acmi_tsv['id'].astype(str)
    acmi_tsv = acmi_tsv.rename(columns={'holdings':'acmi_holdings'})
    acmi_tsv['acmi_holdings'] = acmi_tsv['acmi_holdings'].str.split(',')
    acmi_tsv = acmi_tsv.explode('acmi_holdings')
    acmi_tsv = acmi_tsv[['acmi_id', 'acmi_holdings']].dropna()

    df = sparql_query(query, 'https://query.wikidata.org/sparql').drop_duplicates()
    df['wikidata_id'] = df['wikidata_id'].str.split('/').str[-1]
    df = pandas.merge(df, acmi_tsv, on='acmi_id', how='left').dropna()
    df = df[['wikidata_id', 'acmi_holdings']].drop_duplicates()
    df = df.pivot_table(index=['wikidata_id'], aggfunc=lambda x: ','.join(sorted(x.unique()))).reset_index()

    return df

def nfsa_holdings(wikidata_id):

    ''' Pull NFSA holdings from public API. '''

    query = '''
        select distinct ?wikidata_id ?nfsa_id where {
            ?wikidata_id wdt:P11948 ?nfsa_id
            } '''

    df = sparql_query(query, 'https://query.wikidata.org/sparql').drop_duplicates()
    df['wikidata_id'] = df['wikidata_id'].str.split('/').str[-1]
    df = df.loc[df.wikidata_id.isin(list(wikidata_id))]

    nfsa_dataframe = pandas.DataFrame(columns=['nfsa_id', 'nfsa_holdings'])
    for x in df.nfsa_id.unique():
        r = requests.get(f'https://api.collection.nfsa.gov.au/title/{x}')
        if r.status_code == 200:
            holdings = [y for y in json.loads(r.text)['media']]
            holdings = pydash.uniq([f"{y['itemType']} {y['mediaFormatName']}" for y in holdings])
            nfsa_dataframe.loc[len(nfsa_dataframe)] = [(x), (','.join(holdings))]
        else:
            raise Exception('NFSA API failure.')

    df = pandas.merge(df, nfsa_dataframe, on='nfsa_id', how='left')
    df = df[['wikidata_id', 'nfsa_holdings']]

    return df

dataframe = pandas.read_csv(pathlib.Path.cwd() / 'pike_cooper.csv')
dataframe = pandas.merge(dataframe, acmi_holdings(dataframe.wikidata_id.unique()), on='wikidata_id', how='left')
dataframe = pandas.merge(dataframe, nfsa_holdings(dataframe.wikidata_id.unique()), on='wikidata_id', how='left')
dataframe = dataframe.fillna('')

for x in [x for x in dataframe.to_dict('records') if x['pike_cooper_id'] == 223]:
    for y in ['acmi_holdings', 'nfsa_holdings']:
        x[y] = x[y].split(',')

    print(json.dumps(x, indent=4, ensure_ascii=False))

{
    "pike_cooper_id": 223,
    "year": 1926,
    "title": "The Moth Of Moonbi",
    "director": "Charles Chauvel",
    "wikidata_id": "Q7752407",
    "acmi_holdings": [
        "16mm film; Access Print (Section 1)"
    ],
    "nfsa_holdings": [
        "Film B&W Picture Dupe Negative B-Type",
        "Film B&W Silent Release Print A-Type",
        "Film B&W Silent Release Print B-Type",
        "Tape (Moving Image) 1\" C Format",
        "None Quicktime Movie [ARC]",
        "Film B&W Silent Release Print Full Frame",
        "Film B&W Picture Dupe Negative Full Frame",
        "Film B&W Picture Dupe Negative - Interneg",
        "Film B&W Silent Release Print",
        "Disc (Moving Image) DVD : Digital Versatile Disk",
        "Film B&W Picture Master Positive Full Frame",
        "Disc (Moving Image) None",
        "None Windows Media Video"
    ]
}
